In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tensorflow.keras import utils
from keras.optimizers import adam
from keras.models import Sequential, Model
from keras.layers import Input, InputLayer, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose


def train_ae(encoder, decoder, data, epochs, batch, learning_rate=0.01):
    autoencoder = Sequential([encoder, decoder])
    opt = adam(lr=learning_rate) 
    #autoencoder.summary()
    autoencoder.compile(optimizer=opt, loss='mean_squared_error')
    autoencoder.fit(data, data, epochs=epochs, batch_size=batch)

    data = encoder.predict(data)  
    out_size = data.shape[1:]
    return encoder, data, out_size

def conv_ae(train_images, train_labels, test_images, test_labels, epochs, batch):

    in_shape = train_images[0].shape#(28,28,1)
    cur_data = train_images.copy() 
    encoder_1 = Sequential()
    encoder_1.add(Conv2D(32, (3, 3), activation='sigmoid', padding='same', input_shape=in_shape))
    encoder_1.add(MaxPooling2D(pool_size=(2, 2)))
    decoder_1 = Sequential([Conv2DTranspose(1, (3, 3), strides=(2, 2), padding='same', input_shape=(14, 14, 32))])
    
    encoder_1, cur_data, in_shape = train_ae(encoder_1, decoder_1, cur_data, epochs, batch)
    cur_data = cur_data.reshape(60000, 6272)  
    print('\n')
    encoder_2 = Sequential([Dense(512, activation='sigmoid')])
    decoder_2 = Sequential([Dense(6272, activation='sigmoid')])

    encoder_2, cur_data, in_shape = train_ae(encoder_2, decoder_2, cur_data, epochs, batch)
    print('\n')
    model = Sequential([encoder_1, Flatten(), encoder_2]) 
    model.add(Dense(10, activation='softmax'))

    opt = adam(lr=0.01) 
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    start = time()
    model.fit(train_images, train_labels, epochs=10, batch_size=128)
    train_time = time() - start    
    print('\nCNN Train time ',train_time)
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    print('\nCNN Test loss:', test_loss)
    print('\nCNN Test accuracy:', test_acc)

def main():    
    
    #созд экземпляры слоев, применяем их, объединив в модель
    #load db
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_images = train_images.reshape(60000, 28, 28, 1)
    test_images = test_images.reshape(10000, 28, 28, 1)
 
    train_labels = utils.to_categorical(train_labels, 10)
    test_labels = utils.to_categorical(test_labels, 10)
    #SCALE THIS data DURING 0 AND 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    epoch, batch = 5, 128
    conv_ae(train_images, train_labels, test_images, test_labels, epoch, batch)  
    
if __name__ == '__main__':
    main()

Epoch 1/5
60000/60000 [==============================] - 16s 263us/step - loss: 0.0190
Epoch 2/5
60000/60000 [==============================] - 15s 247us/step - loss: 0.0056
Epoch 3/5
60000/60000 [==============================] - 14s 241us/step - loss: 0.0047
Epoch 4/5
60000/60000 [==============================] - 15s 244us/step - loss: 0.0038
Epoch 5/5
60000/60000 [==============================] - 15s 243us/step - loss: 0.0032


Epoch 1/5
60000/60000 [==============================] - 48s 793us/step - loss: 0.0094
Epoch 2/5
60000/60000 [==============================] - 46s 763us/step - loss: 0.0088
Epoch 3/5
60000/60000 [==============================] - 46s 761us/step - loss: 0.0089
Epoch 4/5
60000/60000 [==============================] - 46s 764us/step - loss: 0.0089
Epoch 5/5
60000/60000 [==============================] - 46s 766us/step - loss: 0.0088


Model: "sequential_74"
_________________________________________________________________
Layer (type)                 Output S